In [11]:
import numpy as np, sys
import pandas as pd
from keras.datasets import mnist

np.random.seed(1)

(x_train, y_train), (x_test, y_test) = mnist.load_data()

images, labels = (x_train[0:1000].reshape(1000,28*28)/255, y_train[0:1000])
#$$(is,ls) =
one_hot_labels = np.zeros((len(labels),10))
for i,l in enumerate(labels):
    one_hot_labels[i][l] = 1
labels = one_hot_labels

test_images = x_test.reshape(len(x_test),28*28) / 255
test_labels = np.zeros((len(y_test),10))
for i, l in enumerate(y_test):
    test_labels[i][l] = 1

def tanh(x):
    return np.tanh(x)

def tanh2deriv(output):
    return 1-(output**2)

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

alpha, iterations = (2,300)
pixels_per_image, num_labels = (784, 10)
batch_size = 128

input_rows = 28
input_cols = 28

kernel_rows = 3
kernel_cols = 3
num_kernels = 16

hidden_size = ((input_rows - kernel_rows) * (input_cols - kernel_cols)) * num_kernels

kernels = 0.02*np.random.random((kernel_rows*kernel_cols,num_kernels)) - 0.01

weights_1_2 = 0.2*np.random.random((hidden_size, num_labels)) - 0.1

def get_image_section(layer,row_from, row_to, col_from, col_to):
    sub_section = layer[:,row_from:row_to,col_from:col_to]
    return sub_section.reshape(-1,1,row_to-row_from, col_to-col_from)

for j in range(iterations):
    correct_cnt = 0
    for i in range(int(len(images) / batch_size)):
        batch_start, batch_end = ((i * batch_size), ((i+1)*batch_size))
        layer_0 = images[batch_start:batch_end]
        layer_0 = layer_0.reshape(layer_0.shape[0],28,28)
        layer_0.shape

        sects = list()
        for row_start in range(layer_0.shape[1] - kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_cols):
                sect = get_image_section(layer_0,row_start,row_start+kernel_rows,col_start,col_start + kernel_cols)
                sects.append(sect)

        expanded_input = np.concatenate(sects,axis=1)
        es = expanded_input.shape
        flattened_input = expanded_input.reshape(es[0]*es[1], -1)
        kernel_output = flattened_input.dot(kernels)
        layer_1 = tanh(kernel_output.reshape(es[0], -1))
        dropout_mask = np.random.randint(2,size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        layer_2 = softmax(np.dot(layer_1,weights_1_2))
        for k in range(batch_size):
            labelset = labels[batch_start+k:batch_start+k+1]
            _inc = int(np.argmax(layer_2[k:k+1]) == np.argmax(labelset))
            correct_cnt += _inc
        layer_2_delta = (labels[batch_start:batch_end] - layer_2) / (batch_size * layer_2.shape[0])
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * tanh2deriv(layer_1)
        layer_1_delta *= dropout_mask * 2
        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        l1d_reshape = layer_1_delta.reshape(kernel_output.shape)
        k_update = flattened_input.T.dot(l1d_reshape)
        kernels -= alpha * k_update
            
    test_correct_cnt = 0

    for i in range(len(test_images)):

        layer_0 = test_images[i:i+1]
        layer_0 = layer_0.reshape(layer_0.shape[0], 28, 28)
        layer_0.shape

        sects = list()
        for row_start in range(layer_0.shape[1] - kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_cols):
                sect = get_image_section(layer_0,row_start,row_start+kernel_rows,col_start, col_start+kernel_cols)
                sects.append(sect)

        expanded_input = np.concatenate(sects,axis=1)
        es = expanded_input.shape
        flattened_input = expanded_input.reshape(es[0]*es[1],-1)

        kernel_output = flattened_input.dot(kernels)
        layer_1 = tanh(kernel_output.reshape(es[0],-1))
        layer_2 = np.dot(layer_1,weights_1_2)

        test_correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i:i+1]))

    if(j % 1 == 0):
        sys.stdout.write("\n" + "I:" + str(j) + " Test-Acc:"+str(test_correct_cnt/float(len(test_images))) + " Train-Acc:" + str(correct_cnt/float(len(images))))




I:0 Test-Acc:0.0397 Train-Acc:0.063
I:1 Test-Acc:0.043 Train-Acc:0.039
I:2 Test-Acc:0.0491 Train-Acc:0.05
I:3 Test-Acc:0.0588 Train-Acc:0.059
I:4 Test-Acc:0.0619 Train-Acc:0.056
I:5 Test-Acc:0.0585 Train-Acc:0.066
I:6 Test-Acc:0.0573 Train-Acc:0.059
I:7 Test-Acc:0.0601 Train-Acc:0.065
I:8 Test-Acc:0.0714 Train-Acc:0.058
I:9 Test-Acc:0.0932 Train-Acc:0.094
I:10 Test-Acc:0.1082 Train-Acc:0.111
I:11 Test-Acc:0.1063 Train-Acc:0.104
I:12 Test-Acc:0.0843 Train-Acc:0.089
I:13 Test-Acc:0.0439 Train-Acc:0.061
I:14 Test-Acc:0.0393 Train-Acc:0.055
I:15 Test-Acc:0.0474 Train-Acc:0.037
I:16 Test-Acc:0.0668 Train-Acc:0.069
I:17 Test-Acc:0.0923 Train-Acc:0.074
I:18 Test-Acc:0.1258 Train-Acc:0.087
I:19 Test-Acc:0.1313 Train-Acc:0.107
I:20 Test-Acc:0.1111 Train-Acc:0.088
I:21 Test-Acc:0.0963 Train-Acc:0.122
I:22 Test-Acc:0.1199 Train-Acc:0.087
I:23 Test-Acc:0.1428 Train-Acc:0.119
I:24 Test-Acc:0.1656 Train-Acc:0.117
I:25 Test-Acc:0.1702 Train-Acc:0.122
I:26 Test-Acc:0.106 Train-Acc:0.121
I:27 Test-Acc